In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
# Cargar el dataset
from utils.lsa64.dataset import LSA64Dataset
from torch.utils.data import DataLoader, random_split
import torch

dataset = LSA64Dataset("../../data/LSA64/landmarks")
torch.manual_seed(42)

train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_ds, test_ds = random_split(dataset, [train_size, test_size])

In [8]:
from torch.nn.utils.rnn import pad_sequence
# Cargarlo al GPU, random_split ya lo shufflea, padearlo con 0s asi tienen el mismo tamaño(al entrenar se ignoran esos 0s)
def collate_pad(batch):
    xs, ys = zip(*batch)
    lengths = torch.tensor([x.size(0) for x in xs])
    x_padded = pad_sequence(xs, batch_first=True, padding_value=0.0)
    y_tensor = torch.tensor(ys)
    return x_padded, lengths, y_tensor

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=6, collate_fn=collate_pad) # Shufflearlo por cada epoch
test_loader = DataLoader(test_ds, batch_size=32, num_workers=6, collate_fn=collate_pad)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [9]:
print(print(len(train_ds), len(test_ds)))

2548 638
None


In [10]:
from models import SimpleRNN
from models import training_loop

In [11]:
model = SimpleRNN()
training_loop(model, train_loader, test_loader, "best_params.pth", lr=1e-4, weight_decay=1e-3)

Epoch 1/100 | Train Acc: 3.12% | Test Acc: 5.96%
Epoch 2/100 | Train Acc: 11.25% | Test Acc: 7.52%
Epoch 3/100 | Train Acc: 11.88% | Test Acc: 8.15%
Epoch 4/100 | Train Acc: 13.75% | Test Acc: 10.82%
Epoch 5/100 | Train Acc: 13.75% | Test Acc: 9.72%
Epoch 6/100 | Train Acc: 13.12% | Test Acc: 9.25%
Epoch 7/100 | Train Acc: 16.25% | Test Acc: 12.07%
Epoch 8/100 | Train Acc: 18.75% | Test Acc: 15.05%
Epoch 9/100 | Train Acc: 16.25% | Test Acc: 14.89%
Epoch 10/100 | Train Acc: 17.50% | Test Acc: 14.11%
Epoch 11/100 | Train Acc: 27.50% | Test Acc: 18.03%
Epoch 12/100 | Train Acc: 22.50% | Test Acc: 17.40%
Epoch 13/100 | Train Acc: 17.50% | Test Acc: 19.12%
Epoch 14/100 | Train Acc: 22.50% | Test Acc: 18.34%
Epoch 15/100 | Train Acc: 23.12% | Test Acc: 19.44%
Epoch 16/100 | Train Acc: 22.50% | Test Acc: 23.67%
Epoch 17/100 | Train Acc: 31.25% | Test Acc: 30.09%
Epoch 18/100 | Train Acc: 43.75% | Test Acc: 31.19%
Epoch 19/100 | Train Acc: 39.38% | Test Acc: 31.19%
Epoch 20/100 | Train Acc: 3

Highest Train Acc: 100.00% (Test Acc at that time: 95.45%)

Highest Test Acc:  98.27% (Train Acc at that time: 100.00%)

# Resultados
- RNN simple, tanh, 5 capas ocultas y 1 de salida, adam, lr=1e-4, lambda=1e-3: 99.38%, 89.94%
  -  Alrededor del doble de datos: 95%, 94.19%

Generaliza para el culo en el mundo real, no es capaz de predecir ni una seña. Sospecho porque en los videos estan todos sentados de la misma forma. Lo que quiero hacer es
- Sacar los landmarks de el torso. Solo tener los landmarks de los brazos y la mano
- Hacer una primera RNN que se dedique a determinar si es una seña de ambas manos o si es de solo la habil(derecha o izquierda, dataset augmentation para zurdos)
- Hacer una segunda etapa dedicada a determinar la seña
  - Una unica RNN con un filtro en su input y reemplazando la mano no-usada con ceros, transfer learning pero quizas le cueste generalizar
  - 2 RNNs diferentes. Una para ambas manos, otra para la mano habil, quizas generalize mejor pero no hay transfer learning

# Ideas para probar
- **REVISAR QUE MODELOS Y ARQUITECTURAS QUE HACEN ESTO YA EXISTEN**
- La idea de PCA para dataset augmentation
  - Agregar vectores de direccion aleatorios a la muñeca/nudillo y mover el resto del brazo con kinesis
- Transfer learning usando otro lenguaje de señas similar al argentino pero mas completo
-   Freezar las conexiones recurrentes? Freezar las primeras capas?
- Un par de capas ocultas mas
- Un par de capas ocultas no recurrentes con ReLU
- Usar una GRU o echo state network para mayor memoria a largo plazo(me parece al pedo una LSTM porque si ya con este nivel de memoria se maneja bien, entonces no creo necesitar tanto control)
- Encontrar una representacion menos ruidosa para los datos(el z lo infiere mal mediapipe en teoria, y la escala de las manos interpoladas tienen mucho ruido)
- Quizas reducir los fps a 6 o menos
- Aumentar el tamaño de los batches
- Reducir el learning rate a $10^{-4}$ una vez que llegue ~80% porque oscila mucho
  - Fijarme en un grafico el punto donde empieza a oscilar
  - No graficar simplemente el train error, graficar la diferencia porcentual entre el train error anterior y este

In [8]:
# Recargar el mejor modelo
model.load_state_dict(torch.load("best_params.pth", map_location=device))

/tmp/ipykernel_719535/2149482387.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_params.pth", map_location=device))


<All keys matched successfully>

In [10]:
# Ahora guardar la softmax de la seña correcta para cada seña en un csv, asi hago analisis despues
labels = []
softmax_score = []

model.eval()
with torch.no_grad():
    for loader in (train_loader, test_loader):
        print("a")
        for x, lengths, y in loader:
            x = x.to(device)
            y = y.to(device)

            y_pred = model(x, lengths)
            _, predicted = y_pred.max(1)
            probs = torch.softmax(y_pred, dim=1)

            for i in range(len(predicted)):
                if int(y[i]) == int(predicted[i]):
                    labels.append(y[i].item())
                    softmax_score.append(probs[i][y[i]].item())

a
a


In [11]:
import pandas as pd
df = pd.DataFrame({"label": labels, "softmax_score": softmax_score})
df.to_csv("rnn_scores.csv", index=False)